This notebook contains the ctu data extraction and QA. It also adds a new UNIQUE_ID field to be used. The working directory needs to be defined in the second cell.

In [1]:
import requests
import arcpy
import zipfile

In [4]:
### Define working directory
wrkdir = r"C:\Users\aoheg\Desktop\School\GIS 5572\Lab2"

In [5]:
# Call API
ctu_url = "https://gisdata.mn.gov/api/3/action/package_show?id=bdry-mn-city-township-unorg"
ctu_response = requests.get(ctu_url)
ctu_data_url = ctu_response.json()['result']['resources'][3]['url']
ctu_data_response = requests.get(ctu_data_url)

# Write data to zip file
f = open(wrkdir + r"\MNctu.zip", "wb")
f.write(ctu_data_response.content)
f.close

# Extract zipfile
with zipfile.ZipFile(wrkdir + r"\MNctu.zip", "r") as zip_ref:
    zip_ref.extractall(wrkdir + r"\MNctu")

In [ ]:
# Check 2 - check for negative or too large populations
arcpy.management.Delete(r"memory\ctu_filter1")
arcpy.management.MakeFeatureLayer(
    wrkdir + r"\MNctu\city_township_unorg.shp",
    r"memory\ctu_filter1",
    where_clause = "POPULATION <= 500000 and POPULATION >= 0"
)

<Result 'memory\\ctu_filter1'>

In [9]:
# Sort and add new id field
arcpy.management.Delete(r"memory\sorted_ctus")
arcpy.management.Sort(r"memory\ctu_filter1", r"memory\sorted_ctus", "FEATURE_NA")
arcpy.management.AddField(r"memory\sorted_ctus", "UNIQUE_ID", "LONG")
with arcpy.da.UpdateCursor(r"memory\sorted_ctus", ["UNIQUE_ID"]) as cursor:
    count = 0 
    for row in cursor:
        row[0] = count
        cursor.updateRow(row)
        count += 1

In [ ]:
# From here it should be written up to the Database